Threshold for Level 1 is 0.36

clf = RandomForestClassifier(n_estimators=100)

Threshold for Level 2 is 0.69

clf = RandomForestClassifier(n_estimators=50,bootstrap= False, criterion = 'gini', max_depth =  3)

Threshold for Level 3 

['AU', 'CA', 'DE', 'ES', 'FR', 'GB', 'IT', 'NL', 'PT', 'other']
[0.013,0.051,0.025,0.078,0.168,0.0761,0.10,0.023,0.002, 0.434]

from sklearn.multiclass import OneVsRestClassifier

params = {'n_estimators': 100, 'learning_rate': 0.1, 'verbose':1, 'random_state':42} 

clf =  OneVsRestClassifier(GradientBoostingClassifier(**params))

In [44]:
import pandas as pd
from sklearn.utils import shuffle
from collections import defaultdict
import numpy as np

In [2]:
#Load in Data
train_users_path = '../../Data/train_users_2.csv'
train_users = pd.read_csv(train_users_path)

sessions_path = '../../Data/sessions.csv'
sessions = pd.read_csv(sessions_path)

test_users_path = '../../Data/test_users.csv'
test_users = pd.read_csv(test_users_path)

In [9]:
final_train_users = train_users[train_users['id'].drop_duplicates().isin(sessions['user_id'].drop_duplicates())]
final_train_users = final_train_users.reset_index()
actions = sessions['action'].dropna().drop_duplicates()

def action_bool(action):
    user_action = sessions[sessions['action'] == action]
    performed = final_train_users['id'].isin(user_action['user_id'])
    colname = 'b_' + action
    final_train_users[colname] = performed

    
for action in actions:
    action_bool(action)
feats = map(lambda x: 'b_' + x,actions)
X = final_train_users[feats + ['country_destination']]
Xsh = shuffle(X,random_state = 42).reset_index()
Xsh = Xsh.as_matrix()[:,1:]	
ntrain = len(Xsh)

Xtrain1 = Xsh[:ntrain,:-1]
ytrain1 = np.ravel(Xsh[:ntrain,-1:] != 'NDF')


In [11]:
print Xtrain1.shape
print ytrain1.shape

(73815, 359)
(73815,)


In [17]:
final_test_users = test_users[test_users['id'].drop_duplicates().isin(sessions['user_id'].drop_duplicates())]
final_test_users = final_test_users.reset_index()

def action_bool(action):
    user_action = sessions[sessions['action'] == action]
    performed = final_test_users['id'].isin(user_action['user_id'])
    colname = 'b_' + action
    final_test_users[colname] = performed

    
for action in actions:
    action_bool(action)
feats = map(lambda x: 'b_' + x,actions)
X = final_test_users[feats]
Xsh = X.as_matrix()[:,:]
ntest = len(Xsh)
Xsh = X.as_matrix()[:,:]
ntest = len(Xsh)

Xtest1 = Xsh[:ntest,:]


In [30]:
print Xtest1.shape

(61668, 359)


In [20]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
clf1 = RandomForestClassifier(n_estimators=100)

clf1.fit(Xtrain1,ytrain1)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [131]:
final_pred1 = defaultdict(bool)
thresh = 0.36
for i in test_users['id']:
    if i in set(final_test_users['id']):
        X = final_test_users[final_test_users['id'] == i][feats]
        Xtest1 = X.as_matrix()
    
        pred1 = clf1.predict_proba(Xtest1)[:,1] >= thresh
        final_pred1[i] = pred1[0]
    else:
        final_pred1[i] = True

KeyboardInterrupt: 

In [126]:
print sum(final_pred1.values())
print len(final_pred1.values())

62096
62096


In [ ]:
clf = RandomForestClassifier(n_estimators=50,bootstrap= False, criterion = 'gini', max_depth =  3)

clf.fit(Xtrlvl2,ytrlvl2)

In [ ]:
#Predict for Level 2